# Session 1 Hands-on Lab: Palawan Land Cover Classification with Random Forest

**Duration:** 90 minutes (1.5 hours)

**Instructor:** CoPhil Advanced Training

---

## Learning Objectives

By the end of this hands-on lab, you will be able to:

1. **Set up and authenticate** Google Earth Engine (GEE) in Google Colab
2. **Acquire and preprocess** Sentinel-2 satellite imagery for a Philippine study area
3. **Calculate spectral indices** (NDVI, NDWI, NDBI, EVI) for land cover discrimination
4. **Create training datasets** by defining land cover classes and sampling spectral signatures
5. **Train a Random Forest classifier** using GEE's machine learning capabilities
6. **Perform land cover classification** on Sentinel-2 imagery
7. **Assess accuracy** using confusion matrices and statistical metrics
8. **Generate area statistics** for each land cover class
9. **Export results** for further analysis and visualization
10. **Apply classification** to real-world Natural Resource Management (NRM) challenges in the Philippines

---

## Study Area: Palawan Province

**Why Palawan?**

- **UNESCO Biosphere Reserve**: Home to exceptional biodiversity and endemic species
- **Conservation Priority**: Contains the Puerto Princesa Subterranean River National Park (UNESCO World Heritage Site)
- **Environmental Challenges**: Deforestation, mining, agricultural expansion, tourism impacts
- **NRM Relevance**: Critical for monitoring forest cover, mangroves, agricultural land conversion
- **Policy Context**: Palawan Strategic Environmental Plan (SEP) requires regular land cover monitoring

Palawan represents a critical case study where accurate land cover classification directly supports conservation planning and sustainable development decisions.

---

## Notebook Overview

This notebook guides you through a complete supervised classification workflow using Random Forest:

```
Data Acquisition → Preprocessing → Feature Engineering → Training Data → Model Training → Classification → Validation → Export
```

**Key Concepts Covered:**
- Cloud computing for Earth Observation (Google Earth Engine)
- Spectral signatures and feature extraction
- Supervised machine learning (Random Forest)
- Model explainability (feature importance)
- Accuracy assessment and validation

---

## A. Setup and Authentication (10 minutes)

### Installing Required Libraries

We'll use:
- **earthengine-api**: Core GEE Python API
- **geemap**: Interactive mapping and visualization
- **pandas**: Data manipulation
- **matplotlib/seaborn**: Visualization
- **numpy**: Numerical operations

In [ ]:
# Install required packages (run only once)
!pip install earthengine-api geemap pandas numpy matplotlib seaborn -q

In [ ]:
# Import libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")
print(f"Earth Engine API version: {ee.__version__}")
print(f"Geemap version: {geemap.__version__}")

### Authenticating with Google Earth Engine

💡 **Understanding GEE Authentication:**

Google Earth Engine requires authentication to access its computational resources and satellite data archives. The process involves:

1. **Authenticate**: Links your Google account to GEE
2. **Initialize**: Connects your Python session to GEE servers

**First-time users:** You'll be redirected to a browser to grant permissions.

**Returning users:** Authentication tokens are cached automatically.

In [ ]:
# Authenticate and initialize Earth Engine
import ee
import geemap.core as geemap

ee.Authenticate()
ee.Initialize(project='YOUR-PROJECT-ID')

⚠️ **Common Issues:**

- **Error: "Unable to authenticate"**: Clear browser cookies or try incognito mode
- **Error: "Project not registered"**: Ensure you've registered your Google Cloud project with Earth Engine
- **Error: "User memory limit exceeded"**: Wait a few hours or optimize your code to reduce memory usage

### Testing the Connection

Let's verify our connection works by querying a simple dataset.

In [ ]:
# Test connection with a simple query
# Instead of a specific image, we'll query a well-known dataset
test_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2025-01-01', '2025-12-31') \
    .first()

# Get basic info
try:
    collection_info = test_collection.getInfo()
    print("✓ Connection successful!")
    print(f"Test Image ID: {collection_info['id']}")
    print(f"Available bands: {len(collection_info['bands'])} bands")
except Exception as e:
    print(f"⚠️ Connection test failed: {str(e)}")
    print("Please check your authentication and try again")

### Creating an Interactive Map

We'll create an interactive map centered on Palawan Province using geemap.

In [ ]:
# Create interactive map centered on Palawan
# Palawan center coordinates: approximately 10.5°N, 118.8°E
Map = geemap.Map(center=[10.5, 118.8], zoom=8, height='600px')

# Add basemap options (SATELLITE basemap)
# Note: basemap is already set by default, or you can use addLayer for custom layers

print("✓ Interactive map created")
print("Use the map controls to pan, zoom, and explore Palawan")
Map

---

## B. Study Area Definition (5 minutes)

We'll define the Palawan province boundary to clip our analysis to the region of interest.

💡 **Understanding Administrative Boundaries:**

For Philippine administrative boundaries, we have several options:

1. **FAO GAUL Dataset**: Global Administrative Unit Layers (Level 0-2)
2. **Manual geometry**: Define coordinates manually
3. **PhilGIS**: Philippine GIS Data Clearinghouse (if imported to GEE)

We'll use the FAO GAUL dataset, which includes Philippine provincial boundaries.

In [ ]:
# Define Palawan boundary using FAO GAUL dataset
# Level 1 = Provincial level in the Philippines
philippines = ee.FeatureCollection('FAO/GAUL/2015/level1')

# Filter for Palawan province
# Note: Palawan may be listed as "Palawan" or subdivided
palawan = philippines.filter(ee.Filter.eq('ADM1_NAME', 'Palawan'))

# Alternative: Define approximate boundary manually if GAUL filter doesn't work
# Uncomment these lines if needed:
# palawan_coords = [
#     [117.5, 9.5], [117.5, 12.0], [119.5, 12.0], [119.5, 9.5], [117.5, 9.5]
# ]
# palawan = ee.Geometry.Polygon(palawan_coords)

print("✓ Palawan boundary defined")

In [ ]:
# Visualize the boundary on the map
Map.addLayer(palawan, {'color': 'red'}, 'Palawan Boundary')
Map.centerObject(palawan, 8)

# Display basic information
area_km2 = palawan.geometry().area().divide(1e6).getInfo()
bounds = palawan.geometry().bounds().getInfo()

print(f"Study Area: Palawan Province")
print(f"Approximate Area: {area_km2:,.0f} km²")

# Safely extract bounding box coordinates
try:
    if bounds and 'coordinates' in bounds:
        print(f"Bounding Box: {bounds['coordinates']}")
    else:
        print("Bounding Box: Coordinates not available")
except:
    print("Bounding Box: Could not retrieve coordinates")

Map

🔍 **Interpretation:**

- Palawan's total area is approximately 14,649 km²
- The province stretches about 450 km from northeast to southwest
- It includes over 1,700 islands and islets
- The main island contains diverse ecosystems: mountains, rainforests, mangroves, coral reefs

---

## C. Sentinel-2 Data Acquisition (20 minutes)

### Loading Sentinel-2 Surface Reflectance Data

💡 **Understanding Sentinel-2 Products:**

- **COPERNICUS/S2_SR**: Surface Reflectance (Level-2A) - atmospherically corrected
- **COPERNICUS/S2**: Top-of-Atmosphere Reflectance (Level-1C) - raw measurements

We use **S2_SR_HARMONIZED** because atmospheric correction is already applied, making it analysis-ready for land cover classification.

**Key Sentinel-2 Specifications:**
- Revisit time: 5 days (with both satellites)
- Spatial resolution: 10m (RGB, NIR), 20m (Red Edge, SWIR), 60m (coastal, aerosol)
- Spectral bands: 13 bands covering visible to short-wave infrared

In [ ]:
# Define date range for imagery
# Using 2025 data (current year)
start_date = '2025-01-01'
end_date = '2025-12-31'

print(f"Date range: {start_date} to {end_date}")

In [ ]:
# Load Sentinel-2 Surface Reflectance collection
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterBounds(palawan)  # Filter by study area
                 .filterDate(start_date, end_date)  # Filter by date range
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))  # Max 20% cloud cover

# Get collection size
collection_size = s2_collection.size().getInfo()
print(f"✓ Sentinel-2 collection loaded")
print(f"Number of images: {collection_size}")

if collection_size == 0:
    print("⚠️ Warning: No images found. Try relaxing cloud cover filter or expanding date range.")

### Cloud Masking

💡 **Understanding Cloud Masking:**

Clouds obscure the land surface and must be removed for accurate classification. Sentinel-2 includes a **QA60** band with cloud mask information:

- **Bit 10**: Opaque clouds
- **Bit 11**: Cirrus clouds

We use bitwise operations to extract these flags and mask cloudy pixels.

In [ ]:
# Define cloud masking function
def mask_s2_clouds(image):
    """
    Masks clouds and cirrus from Sentinel-2 imagery using the QA60 band.
    
    Args:
        image: ee.Image - Sentinel-2 image
    
    Returns:
        ee.Image - Cloud-masked image
    """
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    # Both flags should be set to zero, indicating clear conditions
    mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))
    
    # Return the masked image, scaled to [0, 1]
    return image.updateMask(mask).divide(10000)

print("✓ Cloud masking function defined")

In [ ]:
# Apply cloud masking to the collection
s2_masked = s2_collection.map(mask_s2_clouds)

print("✓ Cloud masking applied to collection")

### Creating a Composite Image

💡 **Understanding Compositing:**

To create a single analysis-ready image from multiple acquisitions, we use compositing:

- **Median composite**: Reduces noise and cloud artifacts (robust to outliers)
- **Mean composite**: Smoother but sensitive to outliers
- **Percentile composite**: e.g., 25th percentile for dark features, 75th for bright features

We'll use **median** as it's most robust for tropical regions with persistent cloud cover.

In [ ]:
# Create median composite
s2_composite = s2_masked.median().clip(palawan)

# Select bands for visualization and analysis
# B2=Blue, B3=Green, B4=Red, B8=NIR, B11=SWIR1, B12=SWIR2
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
s2_composite = s2_composite.select(bands)

print("✓ Median composite created")
print(f"Selected bands: {bands}")

### Visualizing RGB Composite

In [ ]:
# Define visualization parameters for true color (RGB)
rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.4
}

# Add RGB composite to map
Map.addLayer(s2_composite, rgb_vis, 'Sentinel-2 RGB (True Color)')

print("✓ RGB composite added to map")
Map

### Visualizing False Color Composite

💡 **Understanding False Color Composites:**

False color composites use non-visible bands to highlight specific features:

- **NIR-Red-Green (B8-B4-B3)**: Vegetation appears bright red (healthy vegetation reflects strongly in NIR)
- **SWIR-NIR-Red (B12-B8-B4)**: Useful for geology, soil moisture
- **Agriculture (B11-B8-B2)**: Highlights agricultural areas

We'll use **NIR-Red-Green** as it's excellent for distinguishing vegetation from other land covers.

In [ ]:
# Define visualization parameters for false color (NIR-Red-Green)
false_color_vis = {
    'bands': ['B8', 'B4', 'B3'],  # NIR, Red, Green
    'min': 0.0,
    'max': 0.4,
    'gamma': 1.2
}

# Add false color composite to map
Map.addLayer(s2_composite, false_color_vis, 'Sentinel-2 False Color (NIR-R-G)')

print("✓ False color composite added to map")
print("Red areas = healthy vegetation")
print("Green/brown areas = bare soil, urban")
print("Dark blue/black areas = water")

Map

### ✏️ Exercise 1: Experiment with Different Composites

Try creating and visualizing different composite types:

1. **Mean composite** instead of median
2. **25th percentile** composite (darkest pixels)
3. **75th percentile** composite (brightest pixels)

**Question:** How do these composites differ? Which is best for classification?

In [ ]:
# TODO: Create a mean composite
# s2_mean = s2_masked.mean().clip(palawan).select(bands)

# TODO: Create 25th percentile composite
# s2_p25 = s2_masked.reduce(ee.Reducer.percentile([25])).clip(palawan)

# TODO: Visualize and compare


---

## D. Spectral Indices Calculation (15 minutes)

💡 **Understanding Spectral Indices:**

Spectral indices are mathematical combinations of spectral bands that enhance specific land cover features:

- **NDVI** (Normalized Difference Vegetation Index): Measures vegetation health and density
- **NDWI** (Normalized Difference Water Index): Highlights water bodies
- **NDBI** (Normalized Difference Built-up Index): Identifies urban/built-up areas
- **EVI** (Enhanced Vegetation Index): Improved sensitivity in high biomass regions

These indices improve classification accuracy by providing discriminative features beyond raw spectral bands.

### NDVI: Normalized Difference Vegetation Index

$$NDVI = \frac{NIR - Red}{NIR + Red} = \frac{B8 - B4}{B8 + B4}$$

- **Range**: -1 to +1
- **Interpretation**:
  - > 0.6: Dense vegetation (forests)
  - 0.2 to 0.6: Moderate vegetation (grasslands, agriculture)
  - < 0.2: Bare soil, urban, water

In [ ]:
# Calculate NDVI
ndvi = s2_composite.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Visualization parameters
ndvi_vis = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['blue', 'white', 'green', 'darkgreen']
}

Map.addLayer(ndvi, ndvi_vis, 'NDVI')

print("✓ NDVI calculated and visualized")

### NDWI: Normalized Difference Water Index

$$NDWI = \frac{Green - NIR}{Green + NIR} = \frac{B3 - B8}{B3 + B8}$$

- **Range**: -1 to +1
- **Interpretation**:
  - > 0.3: Water bodies
  - 0 to 0.3: Wetlands, moist soil
  - < 0: Vegetation, dry soil

In [ ]:
# Calculate NDWI
ndwi = s2_composite.normalizedDifference(['B3', 'B8']).rename('NDWI')

# Visualization parameters
ndwi_vis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['brown', 'white', 'lightblue', 'darkblue']
}

Map.addLayer(ndwi, ndwi_vis, 'NDWI')

print("✓ NDWI calculated and visualized")

### NDBI: Normalized Difference Built-up Index

$$NDBI = \frac{SWIR1 - NIR}{SWIR1 + NIR} = \frac{B11 - B8}{B11 + B8}$$

- **Range**: -1 to +1
- **Interpretation**:
  - > 0: Urban/built-up areas
  - < 0: Vegetation, water

In [ ]:
# Calculate NDBI
ndbi = s2_composite.normalizedDifference(['B11', 'B8']).rename('NDBI')

# Visualization parameters
ndbi_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ['green', 'white', 'red', 'darkred']
}

Map.addLayer(ndbi, ndbi_vis, 'NDBI')

print("✓ NDBI calculated and visualized")

### EVI: Enhanced Vegetation Index

$$EVI = 2.5 \times \frac{NIR - Red}{NIR + 6 \times Red - 7.5 \times Blue + 1}$$

$$EVI = 2.5 \times \frac{B8 - B4}{B8 + 6 \times B4 - 7.5 \times B2 + 1}$$

- **Range**: -1 to +1 (typically 0 to 1 for vegetation)
- **Advantages over NDVI**:
  - More sensitive in high biomass regions (tropical forests)
  - Corrects for atmospheric and soil background effects
  - Less prone to saturation

In [ ]:
# Calculate EVI
evi = s2_composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': s2_composite.select('B8'),
        'RED': s2_composite.select('B4'),
        'BLUE': s2_composite.select('B2')
    }
).rename('EVI')

# Visualization parameters
evi_vis = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['blue', 'white', 'lightgreen', 'darkgreen']
}

Map.addLayer(evi, evi_vis, 'EVI')

print("✓ EVI calculated and visualized")

### Creating a Side-by-Side Comparison

Let's visualize all indices together to understand their characteristics.

In [ ]:
# Create a new map for side-by-side comparison
Map_indices = geemap.Map(center=[10.5, 118.8], zoom=9, height='600px')

# Add all indices
Map_indices.addLayer(ndvi, ndvi_vis, 'NDVI')
Map_indices.addLayer(ndwi, ndwi_vis, 'NDWI')
Map_indices.addLayer(ndbi, ndbi_vis, 'NDBI')
Map_indices.addLayer(evi, evi_vis, 'EVI')

# Add layer control
Map_indices.add_layer_control()

print("✓ All spectral indices visualized")
print("Use the layer control (top-right) to toggle indices on/off")

Map_indices

### 🔍 Interpretation Exercise

**Questions to consider:**

1. **Where do you see highest NDVI values?** (Hint: Forest areas in the mountains)
2. **Where is NDWI positive?** (Hint: Rivers, lakes, coastal areas)
3. **Where is NDBI highest?** (Hint: Puerto Princesa City, settlements)
4. **How does EVI compare to NDVI in dense forest areas?**
5. **Which index best discriminates between:**
   - Forest vs. Agriculture?
   - Water vs. Bare soil?
   - Urban vs. Vegetation?

### ✏️ Exercise 2: Calculate Additional Indices

Try calculating these additional indices:

1. **SAVI** (Soil-Adjusted Vegetation Index): 
   $$SAVI = \frac{(NIR - Red) \times (1 + L)}{(NIR + Red + L)}$$
   where L = 0.5 (soil brightness correction factor)

2. **GNDVI** (Green Normalized Difference Vegetation Index):
   $$GNDVI = \frac{NIR - Green}{NIR + Green}$$

In [ ]:
# TODO: Calculate SAVI
# L = 0.5
# savi = ...

# TODO: Calculate GNDVI
# gndvi = ...

# TODO: Visualize both indices


---

## E. Feature Stack Preparation (10 minutes)

💡 **Understanding Feature Engineering:**

Machine learning models require a **feature stack** - a multi-band image where each band is an input feature. For land cover classification, our features include:

1. **Spectral bands**: Raw reflectance values (B2, B3, B4, B8, B11, B12)
2. **Spectral indices**: Derived features (NDVI, NDWI, NDBI, EVI)
3. **Optional**: Texture metrics, topography (elevation, slope), temporal features

More features ≠ always better. We need features that:
- Are **discriminative** (help separate classes)
- Are **uncorrelated** (provide independent information)
- Are **analysis-ready** (no missing values, consistent scale)

In [ ]:
# Create comprehensive feature stack
# Combine spectral bands and spectral indices
feature_stack = (s2_composite
                 .addBands(ndvi)
                 .addBands(ndwi)
                 .addBands(ndbi)
                 .addBands(evi))

# List all feature names
feature_names = feature_stack.bandNames().getInfo()

print("✓ Feature stack created")
print(f"Total features: {len(feature_names)}")
print(f"Feature list: {feature_names}")

In [ ]:
# Verify feature completeness (check for masked/missing values)
# This is important to ensure all features are valid for classification

# Get a sample point to check data availability
sample_point = ee.Geometry.Point([118.8, 10.5])  # Central Palawan
sample_values = feature_stack.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=sample_point,
    scale=10
).getInfo()

print("Sample feature values at test point:")
for feature, value in sample_values.items():
    print(f"  {feature}: {value:.4f}" if value is not None else f"  {feature}: NULL")

# Check for any null features
null_features = [f for f, v in sample_values.items() if v is None]
if null_features:
    print(f"⚠️ Warning: Null values found in {null_features}")
else:
    print("✓ All features have valid values")

---

## F. Training Data Preparation (20 minutes)

💡 **Understanding Training Data:**

Training data is the **most critical component** of supervised classification. The quality of your training data directly determines classification accuracy.

**Key principles:**
1. **Representative**: Samples should cover the full spectral variability within each class
2. **Pure**: Each polygon should contain only one land cover type
3. **Well-distributed**: Samples should be spatially distributed across the study area
4. **Sufficient**: At least 50-100 pixels per class (more for heterogeneous classes)
5. **Balanced**: Similar numbers of samples for each class (avoid class imbalance)

### Defining Land Cover Classes

For Palawan, we'll classify 5 major land cover types relevant to NRM:

| Class ID | Class Name | Description | Color |
|----------|------------|-------------|-------|
| 1 | Forest | Primary and secondary forests | Dark Green |
| 2 | Agriculture | Rice paddies, croplands | Yellow |
| 3 | Water | Rivers, lakes, coastal waters | Blue |
| 4 | Urban | Built-up areas, settlements | Red |
| 5 | Bare Soil | Mining areas, cleared land | Brown |

In [ ]:
# Define class properties
class_info = {
    1: {'name': 'Forest', 'color': '006400'},
    2: {'name': 'Agriculture', 'color': 'FFFF00'},
    3: {'name': 'Water', 'color': '0000FF'},
    4: {'name': 'Urban', 'color': 'FF0000'},
    5: {'name': 'Bare Soil', 'color': '8B4513'}
}

print("Land Cover Classification Scheme:")
for class_id, info in class_info.items():
    print(f"  {class_id}: {info['name']}")

### Creating Training Samples

We'll provide pre-defined training samples for Palawan. In operational workflows, you would:

1. **Visual interpretation**: Use high-resolution imagery to identify pure pixels
2. **Field data**: Ground truth from GPS surveys
3. **Existing maps**: Digitize from authoritative land cover maps
4. **Interactive drawing**: Use map tools to draw polygons

⚠️ **Important**: Training data quality > quantity. 50 well-chosen samples beat 500 poor samples.

In [ ]:
# Option 1: Pre-defined training samples (coordinates-based)
# These are approximate locations - adjust based on visual interpretation

# Forest samples (mountainous interior)
forest_coords = [
    [[118.5, 10.5], [118.5, 10.52], [118.52, 10.52], [118.52, 10.5]],
    [[118.7, 10.8], [118.7, 10.82], [118.72, 10.82], [118.72, 10.8]],
    [[119.0, 11.0], [119.0, 11.02], [119.02, 11.02], [119.02, 11.0]],
    [[118.3, 9.8], [118.3, 9.82], [118.32, 9.82], [118.32, 9.8]],
]

# Agriculture samples (coastal plains)
agriculture_coords = [
    [[118.7, 9.75], [118.7, 9.77], [118.72, 9.77], [118.72, 9.75]],
    [[118.9, 10.2], [118.9, 10.22], [118.92, 10.22], [118.92, 10.2]],
    [[118.55, 10.0], [118.55, 10.02], [118.57, 10.02], [118.57, 10.0]],
]

# Water samples (bays, rivers)
water_coords = [
    [[118.73, 9.73], [118.73, 9.75], [118.75, 9.75], [118.75, 9.73]],  # Puerto Princesa Bay
    [[119.3, 10.5], [119.3, 10.52], [119.32, 10.52], [119.32, 10.5]],  # Coastal water
    [[118.4, 10.3], [118.4, 10.32], [118.42, 10.32], [118.42, 10.3]],
]

# Urban samples (Puerto Princesa, towns)
urban_coords = [
    [[118.74, 9.74], [118.74, 9.76], [118.76, 9.76], [118.76, 9.74]],  # Puerto Princesa City
    [[119.08, 10.82], [119.08, 10.84], [119.10, 10.84], [119.10, 10.82]],  # Taytay
]

# Bare soil samples (mining, cleared)
bare_coords = [
    [[117.95, 9.4], [117.95, 9.42], [117.97, 9.42], [117.97, 9.4]],
    [[118.2, 10.1], [118.2, 10.12], [118.22, 10.12], [118.22, 10.1]],
]

# Convert to ee.FeatureCollection
def create_training_features(coords_list, class_value):
    """Convert coordinate list to ee.FeatureCollection with class label."""
    features = []
    for coords in coords_list:
        polygon = ee.Geometry.Polygon(coords)
        feature = ee.Feature(polygon, {'landcover': class_value})
        features.append(feature)
    return ee.FeatureCollection(features)

forest_fc = create_training_features(forest_coords, 1)
agriculture_fc = create_training_features(agriculture_coords, 2)
water_fc = create_training_features(water_coords, 3)
urban_fc = create_training_features(urban_coords, 4)
bare_fc = create_training_features(bare_coords, 5)

# Merge all training samples
training_polygons = (forest_fc
                     .merge(agriculture_fc)
                     .merge(water_fc)
                     .merge(urban_fc)
                     .merge(bare_fc))

print("✓ Training polygons created")
print(f"Total training polygons: {training_polygons.size().getInfo()}")

### Visualizing Training Polygons

In [ ]:
# Create visualization map for training data
Map_training = geemap.Map(center=[10.5, 118.8], zoom=8, height='600px')

# Add base imagery
Map_training.addLayer(s2_composite, rgb_vis, 'Sentinel-2 RGB')

# Add training polygons by class (color-coded)
for class_id, info in class_info.items():
    class_polygons = training_polygons.filter(ee.Filter.eq('landcover', class_id))
    Map_training.addLayer(
        class_polygons,
        {'color': info['color']},
        f"Training: {info['name']}"
    )

print("✓ Training polygons visualized")
print("Toggle layers to see training samples for each class")

Map_training

### Sampling Spectral Values from Training Areas

💡 **Understanding Sampling:**

We extract pixel values from our training polygons to create the training dataset. Each pixel becomes one training sample with:
- **Features**: Spectral band values and indices
- **Label**: Land cover class

**Key parameters:**
- **scale**: Pixel resolution (10m for Sentinel-2 highest resolution bands)
- **geometries**: Whether to include geometry info (not needed for classification)
- **tileScale**: Increases computation tile size to avoid memory errors

In [ ]:
# Sample training data from feature stack
training_samples = feature_stack.sampleRegions(
    collection=training_polygons,
    properties=['landcover'],
    scale=10,  # 10m resolution
    geometries=False,  # We don't need geometry info
    tileScale=4  # Increase if memory errors occur
)

# Get sample count
sample_count = training_samples.size().getInfo()
print(f"✓ Training samples extracted")
print(f"Total training pixels: {sample_count}")

In [ ]:
# Inspect sample distribution by class
# This helps identify class imbalance

class_counts = {}
for class_id in range(1, 6):
    class_samples = training_samples.filter(ee.Filter.eq('landcover', class_id))
    count = class_samples.size().getInfo()
    class_counts[class_id] = count

print("\nSample distribution by class:")
for class_id, count in class_counts.items():
    class_name = class_info[class_id]['name']
    print(f"  {class_name}: {count} pixels")

# Check for severe imbalance
max_count = max(class_counts.values())
min_count = min(class_counts.values())
imbalance_ratio = max_count / min_count if min_count > 0 else float('inf')

if imbalance_ratio > 3:
    print(f"\n⚠️ Warning: Class imbalance detected (ratio: {imbalance_ratio:.1f}:1)")
    print("Consider adding more samples for underrepresented classes")
else:
    print(f"\n✓ Class distribution is balanced (ratio: {imbalance_ratio:.1f}:1)")

### Exploring Training Data

Let's export a small sample to a DataFrame to inspect the spectral signatures.

In [ ]:
# Get a small sample for exploration (limit to 100 samples for speed)
sample_limit = training_samples.limit(100)
sample_list = sample_limit.getInfo()['features']

# Convert to DataFrame
sample_data = [feature['properties'] for feature in sample_list]
df_samples = pd.DataFrame(sample_data)

print("Sample training data (first 10 rows):")
print(df_samples.head(10))

print("\nFeature statistics by class:")
print(df_samples.groupby('landcover')[['NDVI', 'NDWI', 'B4', 'B8']].mean())

### ✏️ Exercise 3: Add More Training Samples

The provided training samples are minimal. To improve classification:

1. **Visually inspect** the RGB composite and identify pure pixels for each class
2. **Draw additional polygons** using the map drawing tools (or add coordinates)
3. **Focus on underrepresented classes** (e.g., if Bare Soil has fewest samples)

**Tip**: Use the false color composite (NIR-R-G) to better distinguish vegetation classes.

In [ ]:
# TODO: Add more training samples
# Option 1: Add coordinate-based polygons
# additional_forest_coords = [
#     [[lon1, lat1], [lon2, lat2], ...],
# ]
# additional_forest_fc = create_training_features(additional_forest_coords, 1)

# Option 2: Interactive drawing (for local use, not Colab)
# Use Map.draw_features to interactively draw training polygons

# Merge with existing samples
# training_polygons = training_polygons.merge(additional_forest_fc)

# Re-sample
# training_samples = feature_stack.sampleRegions(...)


---

## G. Random Forest Training (20 minutes)

💡 **Understanding Random Forest:**

Random Forest is an **ensemble learning** method that combines multiple decision trees:

**How it works:**
1. Create multiple decision trees (e.g., 100 trees)
2. Each tree is trained on a random subset of data (**bagging**)
3. Each tree considers a random subset of features at each split
4. Final prediction = majority vote across all trees

**Advantages:**
- Handles non-linear relationships
- Resistant to overfitting (with enough trees)
- Provides feature importance (explainability)
- No assumption about data distribution
- Works well with high-dimensional data

**Key hyperparameters:**
- **numberOfTrees**: More trees = more stable, but slower (typical: 50-200)
- **variablesPerSplit**: Features considered at each split (default: sqrt(n) is optimal)
- **minLeafPopulation**: Minimum samples per leaf node (default: 1)
- **bagFraction**: Fraction of data used per tree (default: 0.632)

In [ ]:
# Configure Random Forest classifier
rf_classifier = ee.Classifier.smileRandomForest(
    numberOfTrees=100,         # Number of decision trees
    variablesPerSplit=None,    # Auto: sqrt(number of features)
    minLeafPopulation=1,       # Minimum samples per leaf
    bagFraction=0.632,         # Out-of-bag fraction (default)
    seed=42                    # For reproducibility
)

print("✓ Random Forest classifier configured")
print("Configuration:")
print(f"  - Number of trees: 100")
print(f"  - Variables per split: auto (sqrt of {len(feature_names)} features)")
print(f"  - Min leaf population: 1")
print(f"  - Random seed: 42")

In [ ]:
# Train the classifier
print("Training Random Forest classifier...")
print("This may take 1-2 minutes...")

trained_classifier = rf_classifier.train(
    features=training_samples,
    classProperty='landcover',
    inputProperties=feature_names
)

print("✓ Random Forest training complete")

### Feature Importance Analysis

💡 **Understanding Feature Importance:**

Random Forest can tell us which features contribute most to classification decisions. This provides:

1. **Model explainability**: Understand what the model "sees"
2. **Feature selection**: Identify redundant features
3. **Domain insights**: Validate that important features align with physical understanding

**Importance metric**: How much each feature improves classification accuracy across all trees.

Higher importance = more discriminative feature.

In [ ]:
# Extract feature importance
importance = trained_classifier.explain().get('importance')

# Note: GEE's explain() may not always return importance for smileRandomForest
# We'll create a visualization regardless

try:
    importance_dict = importance.getInfo()
    
    # Create DataFrame for visualization
    df_importance = pd.DataFrame({
        'Feature': list(importance_dict.keys()),
        'Importance': list(importance_dict.values())
    }).sort_values('Importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(df_importance['Feature'], df_importance['Importance'])
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.title('Random Forest Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\nTop 5 most important features:")
    print(df_importance.head(5).to_string(index=False))
    
except Exception as e:
    print("⚠️ Feature importance not available from GEE smileRandomForest")
    print("This is a known limitation of the SMILE RF implementation")
    print("\nExpected important features (based on EO literature):")
    print("  1. NIR band (B8) - vegetation discrimination")
    print("  2. NDVI - vegetation index")
    print("  3. SWIR bands (B11, B12) - moisture, urban")
    print("  4. Red band (B4) - vegetation contrast")
    print("  5. NDWI - water detection")

### 🔍 Interpretation: Feature Importance

**Expected patterns:**

- **NIR (B8)** and **NDVI**: Should be highly important (vegetation vs. non-vegetation)
- **SWIR (B11, B12)**: Important for discriminating soil moisture, urban areas
- **NDWI**: Important for water bodies
- **Red (B4)**: Important for vegetation health
- **Blue (B2)**: Often less important (atmospheric scatter, lower contrast)

**Questions:**
1. Do the most important features align with physical understanding?
2. Are spectral indices more important than raw bands?
3. Which features could potentially be removed without losing accuracy?

### ✏️ Exercise 4: Train with Different Parameters

Experiment with different Random Forest configurations:

1. **Fewer trees** (50): Faster but potentially less stable
2. **More trees** (200): More stable but slower
3. **Different variablesPerSplit**: Try 2, 3, or 4 (instead of auto)

**Question**: How does the number of trees affect training time? Does accuracy improve significantly beyond 100 trees?

In [ ]:
# TODO: Train RF with 50 trees
# rf_50 = ee.Classifier.smileRandomForest(numberOfTrees=50, seed=42)
# trained_rf_50 = rf_50.train(...)

# TODO: Train RF with 200 trees
# rf_200 = ee.Classifier.smileRandomForest(numberOfTrees=200, seed=42)
# trained_rf_200 = rf_200.train(...)

# TODO: Compare results (we'll evaluate accuracy in next section)


---

## H. Image Classification (15 minutes)

Now we'll apply the trained classifier to the entire Palawan image.

In [ ]:
# Apply classifier to feature stack
print("Classifying image...")
print("This may take 1-2 minutes...")

classified_image = feature_stack.classify(trained_classifier)

print("✓ Classification complete")

In [ ]:
# Define color palette for visualization
# Colors match class_info defined earlier
class_palette = [class_info[i]['color'] for i in sorted(class_info.keys())]

print(f"Classification palette: {class_palette}")

In [ ]:
# Visualize classification result
Map_classified = geemap.Map(center=[10.5, 118.8], zoom=8, height='700px')

# Add base imagery for comparison
Map_classified.addLayer(s2_composite, rgb_vis, 'Sentinel-2 RGB', False)
Map_classified.addLayer(s2_composite, false_color_vis, 'False Color', False)

# Add classification
Map_classified.addLayer(
    classified_image,
    {'min': 1, 'max': 5, 'palette': class_palette},
    'Land Cover Classification'
)

# Add training polygons for reference
for class_id, info in class_info.items():
    class_polygons = training_polygons.filter(ee.Filter.eq('landcover', class_id))
    Map_classified.addLayer(
        class_polygons,
        {'color': info['color']},
        f"Training: {info['name']}",
        False  # Hidden by default
    )

# Add layer control
Map_classified.add_layer_control()

print("✓ Classification visualized")
print("\nLegend:")
for class_id, info in class_info.items():
    print(f"  {info['name']}: #{info['color']}")

Map_classified

### 🔍 Visual Interpretation

**Areas to examine:**

1. **Forest coverage**: 
   - Toggle between RGB and classification
   - Are mountainous interior areas classified as forest?
   - Are mangrove forests along coasts detected?

2. **Agriculture**:
   - Check coastal plains and river valleys
   - Are rice paddies correctly identified?

3. **Water bodies**:
   - Rivers, lakes, bays
   - Any confusion with shadows or dark vegetation?

4. **Urban areas**:
   - Puerto Princesa City (main urban center)
   - Small towns along roads

5. **Potential misclassifications**:
   - Cloud shadows misclassified as water?
   - Bare soil confused with urban?
   - Sparse vegetation confused with agriculture?

**Tip**: Zoom to specific locations and toggle layers to compare classification with imagery.

### ✏️ Exercise 5: Identify Misclassifications

Explore the map and identify at least 3 areas where classification appears incorrect:

1. **Describe the misclassification**: What was classified? What should it be?
2. **Hypothesize why**: Spectral confusion? Poor training samples? Class definition?
3. **Propose solution**: Add training data? Refine class definitions? Add features?

**Your observations:**

1. Misclassification 1:
   - Location: [latitude, longitude]
   - Classified as: [class]
   - Should be: [class]
   - Why: ...
   - Solution: ...

2. Misclassification 2:
   - ...

3. Misclassification 3:
   - ...

---

## I. Accuracy Assessment (25 minutes)

💡 **Understanding Accuracy Assessment:**

Visual inspection is subjective. We need **quantitative metrics** to evaluate classification performance:

**Confusion Matrix**: Cross-tabulation of predicted vs. actual classes
- Rows = reference (true) class
- Columns = predicted class
- Diagonal = correct classifications
- Off-diagonal = misclassifications

**Key Metrics**:
1. **Overall Accuracy**: (Correct predictions) / (Total predictions)
2. **Producer's Accuracy**: (Correctly classified samples of class X) / (Total reference samples of class X)
   - Measures omission error (missed detections)
3. **User's Accuracy**: (Correctly classified samples of class X) / (Total predictions of class X)
   - Measures commission error (false alarms)
4. **Kappa Coefficient**: Agreement beyond random chance (0 = random, 1 = perfect)

**Best Practice**: Use **independent validation data** (not used in training). We'll simulate this by splitting our training data.

### Train-Validation Split

We'll split our samples into:
- **Training set (80%)**: Used to train the model
- **Validation set (20%)**: Used to assess accuracy

In [ ]:
# Add random column for splitting
training_samples = training_samples.randomColumn('random', seed=42)

# Split: 80% training, 20% validation
split = 0.8
training_set = training_samples.filter(ee.Filter.lt('random', split))
validation_set = training_samples.filter(ee.Filter.gte('random', split))

# Get counts
train_count = training_set.size().getInfo()
val_count = validation_set.size().getInfo()

print("Data split:")
print(f"  Training samples: {train_count} ({train_count/(train_count+val_count)*100:.1f}%)")
print(f"  Validation samples: {val_count} ({val_count/(train_count+val_count)*100:.1f}%)")

In [ ]:
# Retrain classifier on training set only
print("Retraining classifier on training set...")

rf_final = ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=None,
    minLeafPopulation=1,
    seed=42
)

trained_final = rf_final.train(
    features=training_set,
    classProperty='landcover',
    inputProperties=feature_names
)

print("✓ Retraining complete")

In [ ]:
# Classify validation set
validation_classified = validation_set.classify(trained_final)

print("✓ Validation set classified")

### Confusion Matrix

In [ ]:
# Generate confusion matrix
confusion_matrix = validation_classified.errorMatrix('landcover', 'classification')

# Get matrix as array
matrix_array = confusion_matrix.array().getInfo()

print("Confusion Matrix:")
print("Rows = Reference (True), Columns = Predicted")
print(matrix_array)

In [ ]:
# Visualize confusion matrix as heatmap
class_names = [class_info[i]['name'] for i in sorted(class_info.keys())]

plt.figure(figsize=(10, 8))
sns.heatmap(
    matrix_array,
    annot=True,
    fmt='g',
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'}
)
plt.xlabel('Predicted Class', fontsize=12)
plt.ylabel('Reference Class', fontsize=12)
plt.title('Confusion Matrix - Palawan Land Cover Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Accuracy Metrics

In [ ]:
# Calculate overall accuracy
overall_accuracy = confusion_matrix.accuracy().getInfo()
print(f"Overall Accuracy: {overall_accuracy*100:.2f}%")

# Calculate Kappa coefficient
kappa = confusion_matrix.kappa().getInfo()
print(f"Kappa Coefficient: {kappa:.4f}")

# Interpret Kappa
if kappa > 0.8:
    kappa_interp = "Excellent agreement"
elif kappa > 0.6:
    kappa_interp = "Substantial agreement"
elif kappa > 0.4:
    kappa_interp = "Moderate agreement"
else:
    kappa_interp = "Poor agreement"

print(f"Interpretation: {kappa_interp}")

In [ ]:
# Calculate Producer's Accuracy (per class)
producers_accuracy = confusion_matrix.producersAccuracy().getInfo()

print("\nProducer's Accuracy (Sensitivity, Recall):")
print("Measures: How many reference samples were correctly classified?")
for i, acc in enumerate(producers_accuracy):
    class_name = class_info[i+1]['name']
    print(f"  {class_name}: {acc*100:.2f}%")

In [ ]:
# Calculate User's Accuracy (per class)
users_accuracy = confusion_matrix.consumersAccuracy().getInfo()

print("\nUser's Accuracy (Precision):")
print("Measures: How many predicted samples are actually correct?")
for i, acc in enumerate(users_accuracy):
    class_name = class_info[i+1]['name']
    print(f"  {class_name}: {acc*100:.2f}%")

In [ ]:
# Create summary DataFrame
df_accuracy = pd.DataFrame({
    'Class': class_names,
    'Producer\'s Accuracy (%)': [acc*100 for acc in producers_accuracy],
    'User\'s Accuracy (%)': [acc*100 for acc in users_accuracy]
})

print("\nAccuracy Summary by Class:")
print(df_accuracy.to_string(index=False))

In [ ]:
# Visualize per-class accuracy
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(class_names))
width = 0.35

ax.bar(x - width/2, df_accuracy['Producer\'s Accuracy (%)'], width, label='Producer\'s Accuracy', alpha=0.8)
ax.bar(x + width/2, df_accuracy['User\'s Accuracy (%)'], width, label='User\'s Accuracy', alpha=0.8)

ax.set_xlabel('Land Cover Class', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Per-Class Accuracy Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(class_names)
ax.legend()
ax.set_ylim([0, 105])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 🔍 Analyzing Classification Errors

**Questions to consider:**

1. **Which classes are most accurately classified?**
   - Typically: Water (distinct spectral signature)
   - Look for classes with >90% accuracy

2. **Which classes are confused with each other?**
   - Look at off-diagonal elements in confusion matrix
   - Common confusions:
     - Agriculture ↔ Bare Soil (if crop fields are bare)
     - Forest ↔ Agriculture (if crops have high biomass)
     - Urban ↔ Bare Soil (similar spectral response)

3. **Is there class imbalance affecting results?**
   - Classes with fewer samples often have lower accuracy

4. **How can we improve?**
   - Add more training samples for poorly classified classes
   - Refine class definitions (subdivide confusable classes)
   - Add discriminative features (texture, elevation, temporal)
   - Use multi-temporal data (phenology helps separate agriculture from forest)

### ✏️ Exercise 6: Try Different Train/Test Splits

Experiment with different split ratios:

1. **70-30 split**: More validation data
2. **90-10 split**: More training data

**Question**: How does the split ratio affect accuracy? Is there a trade-off?

In [ ]:
# TODO: Experiment with different splits
# split_70 = 0.7
# training_set_70 = training_samples.filter(ee.Filter.lt('random', split_70))
# validation_set_70 = training_samples.filter(ee.Filter.gte('random', split_70))

# Retrain and evaluate
# ...


---

## J. Area Statistics (10 minutes)

Let's calculate the area covered by each land cover class.

In [ ]:
# Calculate pixel count for each class
# Reclassify to get area (each pixel = 100 m²)

pixel_area = ee.Image.pixelArea()  # Returns area in square meters

# Calculate area per class
area_image = pixel_area.addBands(classified_image)

# Reduce by class
area_stats = area_image.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='landcover'
    ),
    geometry=palawan,
    scale=10,  # 10m resolution
    maxPixels=1e10,
    tileScale=4
)

print("Calculating area statistics...")
print("This may take 2-3 minutes...")

In [ ]:
# Extract area results
area_results = area_stats.getInfo()
area_groups = area_results['groups']

# Convert to DataFrame
area_data = []
for group in area_groups:
    class_id = group['landcover']
    area_m2 = group['sum']
    area_km2 = area_m2 / 1e6
    area_ha = area_m2 / 1e4
    
    area_data.append({
        'Class ID': class_id,
        'Class Name': class_info[class_id]['name'],
        'Area (km²)': area_km2,
        'Area (ha)': area_ha
    })

df_area = pd.DataFrame(area_data).sort_values('Area (km²)', ascending=False)

# Calculate percentages
total_area = df_area['Area (km²)'].sum()
df_area['Percentage (%)'] = (df_area['Area (km²)'] / total_area) * 100

print("\nLand Cover Area Statistics:")
print(df_area.to_string(index=False))
print(f"\nTotal classified area: {total_area:,.0f} km²")

In [ ]:
# Visualize area distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart
colors = [f"#{class_info[row['Class ID']]['color']}" for _, row in df_area.iterrows()]
ax1.bar(df_area['Class Name'], df_area['Area (km²)'], color=colors, alpha=0.7)
ax1.set_xlabel('Land Cover Class', fontsize=12)
ax1.set_ylabel('Area (km²)', fontsize=12)
ax1.set_title('Land Cover Distribution', fontsize=14, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Pie chart
ax2.pie(
    df_area['Percentage (%)'],
    labels=df_area['Class Name'],
    autopct='%1.1f%%',
    colors=colors,
    startangle=90
)
ax2.set_title('Land Cover Percentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 🔍 Interpreting Area Statistics

**Questions:**

1. **What is the dominant land cover?**
   - Expected: Forest (Palawan is ~50-60% forested)

2. **How much agricultural land?**
   - Agriculture is mainly in coastal plains and river valleys

3. **Do results align with official statistics?**
   - Compare with DENR/PhilSA land cover maps
   - Note: Differences expected due to classification scheme, date, methods

4. **What about urban area?**
   - Urban should be small (Palawan is relatively undeveloped)
   - Main urban center: Puerto Princesa City

**Conservation context:**
- Forest cover is critical for biodiversity conservation
- Monitoring deforestation trends requires multi-temporal analysis
- This single-date classification provides a baseline

---

## K. Export Results (10 minutes)

💡 **Understanding GEE Exports:**

Google Earth Engine processing happens on Google's servers. To get results locally, we export to:
- **Google Drive**: Rasters (GeoTIFF), vectors (Shapefile, GeoJSON), tables (CSV)
- **Cloud Storage**: For large files or automated workflows
- **Asset**: For reuse in GEE

**Important**: Exports are **asynchronous tasks**. They don't block notebook execution but run in the background.

### Export Classified Image

In [ ]:
# Export classification to Google Drive
export_classification = ee.batch.Export.image.toDrive(
    image=classified_image,
    description='Palawan_LandCover_Classification',
    folder='EarthEngine',
    fileNamePrefix='palawan_landcover_2025',
    region=palawan.geometry(),
    scale=10,  # 10m resolution
    crs='EPSG:4326',
    maxPixels=1e10,
    fileFormat='GeoTIFF'
)

# Start the export task
export_classification.start()

print("✓ Classification export task started")
print(f"Task description: {export_classification.status()['description']}")
print(f"Task state: {export_classification.status()['state']}")
print("\nCheck Google Drive > EarthEngine folder for the file")
print("Note: Export may take 5-15 minutes depending on area size")

### Export Training Samples

In [ ]:
# Export training samples to Google Drive (CSV)
export_training = ee.batch.Export.table.toDrive(
    collection=training_samples,
    description='Palawan_Training_Samples',
    folder='EarthEngine',
    fileNamePrefix='palawan_training_samples',
    fileFormat='CSV'
)

export_training.start()

print("✓ Training samples export task started")
print(f"Task state: {export_training.status()['state']}")

### Export Confusion Matrix

In [ ]:
# Save confusion matrix locally (as CSV)
df_confusion = pd.DataFrame(
    matrix_array,
    index=class_names,
    columns=class_names
)

# Save to local file (will be available in Colab session)
df_confusion.to_csv('palawan_confusion_matrix.csv')

print("✓ Confusion matrix saved locally")
print("File: palawan_confusion_matrix.csv")
print("\nDownload from Files panel (left sidebar)")

### Export Area Statistics

In [ ]:
# Save area statistics locally (as CSV)
df_area.to_csv('palawan_area_statistics.csv', index=False)

print("✓ Area statistics saved locally")
print("File: palawan_area_statistics.csv")

### Check Export Task Status

In [ ]:
# Check status of export tasks
import time

print("Export task status:")
print(f"  Classification: {export_classification.status()['state']}")
print(f"  Training samples: {export_training.status()['state']}")

print("\nPossible states:")
print("  READY: Task is queued")
print("  RUNNING: Task is processing")
print("  COMPLETED: Task finished successfully")
print("  FAILED: Task encountered an error")
print("  CANCELLED: Task was cancelled")

print("\nTo check status later, visit:")
print("https://code.earthengine.google.com/tasks")

⚠️ **Common Export Issues:**

1. **"User memory limit exceeded"**: Reduce scale (e.g., 30m instead of 10m) or reduce region size
2. **"Computation timed out"**: Add `tileScale` parameter (e.g., `tileScale=4`)
3. **"Too many concurrent operations"**: GEE limits concurrent tasks (wait for others to complete)
4. **File not in Drive**: Check "EarthEngine" folder; exports may take 5-30 minutes

---

## L. Advanced Exercises (Optional, 15 minutes)

For participants who finish early, here are advanced challenges:

### ✏️ Exercise 7: Add More Spectral Indices

Calculate and add these additional indices to the feature stack:

1. **SAVI** (Soil-Adjusted Vegetation Index): Reduces soil background effects
2. **GNDVI** (Green NDVI): Sensitive to chlorophyll concentration
3. **NBR** (Normalized Burn Ratio): Useful for detecting burned areas

$$SAVI = \frac{(NIR - Red) \times (1 + L)}{(NIR + Red + L)}$$ where L = 0.5

$$GNDVI = \frac{NIR - Green}{NIR + Green}$$

$$NBR = \frac{NIR - SWIR2}{NIR + SWIR2}$$

**Question**: Do these additional indices improve classification accuracy?

In [ ]:
# TODO: Calculate SAVI, GNDVI, NBR
# savi = ...
# gndvi = ...
# nbr = ...

# TODO: Add to feature stack
# feature_stack_extended = feature_stack.addBands([savi, gndvi, nbr])

# TODO: Retrain classifier and compare accuracy


### ✏️ Exercise 8: Multi-temporal Analysis

Land cover has temporal patterns (phenology):
- **Forests**: Relatively stable NDVI year-round
- **Agriculture**: Seasonal NDVI variations (planting, growing, harvest)
- **Water**: Stable, low NDVI

Create composites for dry season (Jan-Apr) and wet season (Jun-Sep), then classify.

**Question**: Does multi-temporal data improve discrimination between forest and agriculture?

In [ ]:
# TODO: Create dry season composite
# dry_season = s2_masked.filterDate('2025-01-01', '2025-04-30').median()

# TODO: Create wet season composite
# wet_season = s2_masked.filterDate('2025-06-01', '2025-09-30').median()

# TODO: Calculate NDVI for each season
# ndvi_dry = dry_season.normalizedDifference(['B8', 'B4']).rename('NDVI_dry')
# ndvi_wet = wet_season.normalizedDifference(['B8', 'B4']).rename('NDVI_wet')

# TODO: Create multi-temporal feature stack
# feature_stack_temporal = dry_season.select(bands).addBands(wet_season.select(bands)).addBands([ndvi_dry, ndvi_wet])

# TODO: Train and evaluate


### ✏️ Exercise 9: Add Topographic Features

Elevation and slope can improve classification:
- **Forests**: Often in mountainous areas
- **Agriculture**: Mainly in lowlands and valleys
- **Water**: Low elevation, flat

Add SRTM elevation and derived slope to features.

In [ ]:
# TODO: Load SRTM elevation data
# srtm = ee.Image('USGS/SRTMGL1_003')
# elevation = srtm.select('elevation').clip(palawan)

# TODO: Calculate slope
# slope = ee.Terrain.slope(elevation)

# TODO: Add to feature stack
# feature_stack_topo = feature_stack.addBands([elevation, slope])

# TODO: Train and evaluate


### ✏️ Exercise 10: Classify a Different Region

Apply the same workflow to a different Philippine region:
- **Metro Manila**: Urban-focused classification
- **Ifugao (Rice Terraces)**: Agricultural focus
- **Mindanao**: Different ecological zone

**Question**: Are the same features important? Does the model generalize?

In [ ]:
# TODO: Define new study area
# new_region = ...

# TODO: Load Sentinel-2 data
# ...

# TODO: Create training samples
# ...

# TODO: Train and evaluate


### ✏️ Exercise 11: Compare with CART (Single Decision Tree)

Random Forest is an ensemble of trees. How does a single tree compare?

Train a CART (Classification and Regression Tree) classifier and compare accuracy.

In [ ]:
# TODO: Train CART classifier
# cart_classifier = ee.Classifier.smileCart()
# trained_cart = cart_classifier.train(
#     features=training_set,
#     classProperty='landcover',
#     inputProperties=feature_names
# )

# TODO: Classify and evaluate
# ...

# TODO: Compare confusion matrices and accuracy


---

## Summary and Reflection

### What We Learned

In this hands-on lab, you:

1. ✓ **Set up Google Earth Engine** and authenticated successfully
2. ✓ **Acquired Sentinel-2 data** for Palawan, Philippines
3. ✓ **Preprocessed imagery** using cloud masking and compositing
4. ✓ **Calculated spectral indices** (NDVI, NDWI, NDBI, EVI)
5. ✓ **Created training datasets** with 5 land cover classes
6. ✓ **Trained a Random Forest classifier** with 100 trees
7. ✓ **Performed land cover classification** on the entire study area
8. ✓ **Assessed accuracy** using confusion matrices and statistical metrics
9. ✓ **Generated area statistics** for each land cover class
10. ✓ **Exported results** to Google Drive for further analysis

### Key Takeaways

**Technical:**
- Google Earth Engine enables cloud-based analysis of petabytes of satellite data
- Random Forest is a powerful, interpretable algorithm for land cover classification
- Feature engineering (spectral indices) improves discrimination
- Training data quality is critical - garbage in, garbage out
- Accuracy assessment requires independent validation data

**Application to Philippine NRM:**
- Land cover classification supports conservation planning (e.g., Palawan SEP)
- Regular monitoring enables detection of deforestation, land conversion
- Integration with PhilSA, NAMRIA, DENR datasets enhances operational value
- Scalable workflows can be applied to other Philippine regions

### Next Steps

**To improve this classification:**
1. **Collect more training data**: Field surveys with GPS, visual interpretation
2. **Refine class definitions**: Separate primary/secondary forest, mangroves
3. **Add temporal features**: Multi-season composites capture phenology
4. **Incorporate ancillary data**: Elevation, slope, distance to roads/water
5. **Post-processing**: Spatial filtering to remove isolated pixels

**For operational deployment:**
1. **Automate workflow**: Schedule regular updates (monthly, quarterly)
2. **Change detection**: Compare classifications over time
3. **Integrate with decision systems**: Feed results to monitoring dashboards
4. **Validate with ground truth**: Coordinate with local agencies

### Reflection Questions

1. What surprised you about the classification results?
2. Which land cover classes were easiest/hardest to classify? Why?
3. How would you explain the classification to a non-technical stakeholder?
4. What additional data would improve the classification?
5. How could this workflow support your organization's mission?

---

## Additional Resources

**Google Earth Engine:**
- [GEE Documentation](https://developers.google.com/earth-engine)
- [GEE Code Editor](https://code.earthengine.google.com/)
- [geemap Documentation](https://geemap.org/)

**Sentinel-2:**
- [Sentinel-2 User Handbook](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi)
- [Copernicus Open Access Hub](https://scihub.copernicus.eu/)

**Machine Learning for EO:**
- Gislason et al. (2006): "Random Forests for land cover classification" - *Remote Sensing of Environment*
- Maxwell et al. (2018): "Implementation of machine-learning classification in remote sensing" - *International Journal of Remote Sensing*

**Philippine EO:**
- [PhilSA Space+ Data Dashboard](https://data.philsa.gov.ph/)
- [NAMRIA Geoportal](https://www.geoportal.gov.ph/)
- [DOST-ASTI DATOS](https://datos.asti.dost.gov.ph/)

---

## Troubleshooting

### Common Issues and Solutions

**Issue: "Earth Engine authentication failed"**
- Solution: Run `ee.Authenticate()` and follow the prompts
- Clear browser cache if persistent

**Issue: "Computation timed out"**
- Solution: Add `tileScale=4` to `sampleRegions()` or `reduceRegion()`
- Reduce spatial resolution (e.g., `scale=30` instead of `scale=10`)

**Issue: "User memory limit exceeded"**
- Solution: Reduce study area size or increase `tileScale`
- Export large computations instead of `.getInfo()`

**Issue: "No images in collection"**
- Solution: Relax cloud cover filter (e.g., `<30%` instead of `<20%`)
- Expand date range

**Issue: "Classification has many misclassifications"**
- Solution: Add more training samples
- Check that training polygons are pure (single land cover)
- Add discriminative features

**Issue: "Export task failed"**
- Solution: Check task status at https://code.earthengine.google.com/tasks
- Reduce export region or resolution
- Check Google Drive storage quota

---

**End of Hands-on Lab**

Thank you for participating in this session! Your feedback is valuable - please share any suggestions for improving this lab.

**Contact**: [Training organizer email]

---